<a href="https://colab.research.google.com/github/nakamura196/000_tools/blob/main/NDL%E5%8F%A4%E5%85%B8%E7%B1%8DOCR_v2%E3%81%AE%E5%AE%9F%E8%A1%8C%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NDL古典籍OCRについては、デスクトップアプリケーションとして簡易に利用可能な「NDL古典籍OCR-Lite」が公開されました。今後は、こちらをお使いいただけますと幸いです。

https://github.com/ndl-lab/ndlkotenocr-lite